In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline

import warnings

warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np
# import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import plotly.express as px
from sklearn.linear_model import LogisticRegression
import plotly.graph_objs as go
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import svm


from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier

# from keras.models import Sequential

from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle



In [ ]:
df1=pd.read_csv('weatherAUS.csv')

In [ ]:
df1.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,02-12-2008,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,03-12-2008,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,04-12-2008,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,05-12-2008,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [ ]:
# function for preprocessing data
def preprocess_dataframe(df):
    df_filtered=df[df['RainTomorrow'].notna()]
    df_inference=df[df['RainTomorrow'].isna()]
    return df_filtered, df_inference

In [ ]:
df1.head(1)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No


In [ ]:
df1, df_inference = preprocess_dataframe(df1)

In [ ]:
df1.head(1)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,01-12-2008,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No


In [ ]:
df_inference.head(1)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
14,15-12-2008,Albury,8.4,24.6,0.0,NaN,NaN,NaN,NaN,S,...,57.0,32.0,1009.7,1008.7,NaN,NaN,15.9,23.5,No,NaN


In [ ]:
def process_date_column(df, date_column):
    df[date_column] = pd.to_datetime(df[date_column], format='%d-%m-%Y', dayfirst=True)
    df['year'] = df[date_column].dt.year
    df['month'] = df[date_column].dt.month
    df['day'] = df[date_column].dt.day
    df.drop(date_column, axis=1, inplace=True)
    return df


In [ ]:
# from datetime import datetime

# date_string = '13-12-2008'
# date_format = '%d-%m-%Y'

# date_object = datetime.strptime(date_string, date_format)

# print(date_object)

In [ ]:

df2 = process_date_column(df1, 'Date')


df_inference=process_date_column(df_inference,'Date')

In [ ]:
df2.head(1)

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year,month,day
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,1007.1,8.0,NaN,16.9,21.8,No,No,2008,12,1


In [ ]:
def impute_missing_values(df):
    """Imputes missing values in a DataFrame.

    Args:
        df (pd.DataFrame): The DataFrame to impute missing values in.

    Returns:
        pd.DataFrame: The DataFrame with missing values imputed.
    """
    columns_with_null = df.isnull().sum()
    columns_with_null = columns_with_null[columns_with_null > 0]

    for column in columns_with_null.index:
        if df[column].dtypes == 'object':
            # Check if there are any duplicate values in the column
            if df[column].nunique() > 1:
                mode_value = df[column].mode()[0]
                df[column].fillna(mode_value, inplace=True)
            else:
                # If there are no duplicate values, fill with the only unique value
                df[column].fillna(df[column].unique()[0], inplace=True)
        elif df[column].dtypes == 'float64':
            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)

    return df


In [ ]:
df2=impute_missing_values(df2)
df_inference=impute_missing_values(df_inference)

In [ ]:
df2.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year,month,day
0,Albury,13.4,22.9,0.6,5.469824,7.624853,W,44.0,W,WNW,...,1007.1,8.000000,4.503167,16.9,21.8,No,No,2008,12,1
1,Albury,7.4,25.1,0.0,5.469824,7.624853,WNW,44.0,NNW,WSW,...,1007.8,4.437189,4.503167,17.2,24.3,No,No,2008,12,2
2,Albury,12.9,25.7,0.0,5.469824,7.624853,WSW,46.0,W,WSW,...,1008.7,4.437189,2.000000,21.0,23.2,No,No,2008,12,3
3,Albury,9.2,28.0,0.0,5.469824,7.624853,NE,24.0,SE,E,...,1012.8,4.437189,4.503167,18.1,26.5,No,No,2008,12,4
4,Albury,17.5,32.3,1.0,5.469824,7.624853,W,41.0,ENE,NW,...,1006.0,7.000000,8.000000,17.8,29.7,No,No,2008,12,5


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142193 entries, 0 to 145458
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  object 
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   Evaporation    142193 non-null  float64
 5   Sunshine       142193 non-null  float64
 6   WindGustDir    142193 non-null  object 
 7   WindGustSpeed  142193 non-null  float64
 8   WindDir9am     142193 non-null  object 
 9   WindDir3pm     142193 non-null  object 
 10  WindSpeed9am   142193 non-null  float64
 11  WindSpeed3pm   142193 non-null  float64
 12  Humidity9am    142193 non-null  float64
 13  Humidity3pm    142193 non-null  float64
 14  Pressure9am    142193 non-null  float64
 15  Pressure3pm    142193 non-null  float64
 16  Cloud9am       142193 non-null  float64
 17  Cloud3pm       142193 non-null  fl

In [ ]:
df_inference.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3267 entries, 14 to 145459
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Location       3267 non-null   object 
 1   MinTemp        3267 non-null   float64
 2   MaxTemp        3267 non-null   float64
 3   Rainfall       3267 non-null   float64
 4   Evaporation    3267 non-null   float64
 5   Sunshine       3267 non-null   float64
 6   WindGustDir    3267 non-null   object 
 7   WindGustSpeed  3267 non-null   float64
 8   WindDir9am     3267 non-null   object 
 9   WindDir3pm     3267 non-null   object 
 10  WindSpeed9am   3267 non-null   float64
 11  WindSpeed3pm   3267 non-null   float64
 12  Humidity9am    3267 non-null   float64
 13  Humidity3pm    3267 non-null   float64
 14  Pressure9am    3267 non-null   float64
 15  Pressure3pm    3267 non-null   float64
 16  Cloud9am       3267 non-null   float64
 17  Cloud3pm       3267 non-null   float64
 18  Temp9am   

In [ ]:
df2.columns

Index(['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday', 'RainTomorrow', 'year', 'month', 'day'],
      dtype='object')

In [ ]:
df2.isnull().sum()

,0
Location,0
MinTemp,0
MaxTemp,0
Rainfall,0
Evaporation,0
Sunshine,0
WindGustDir,0
WindGustSpeed,0
WindDir9am,0
WindDir3pm,0


In [ ]:
"""import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

def fit_label_encoders(df: pd.DataFrame, save_path: str) -> dict:
    label_encoders = {}
    for col in df.columns:
        if df[col].dtype == 'object' and col != 'RainTomorrow':
            le = LabelEncoder()
            le.fit(df[col].astype(str))
            label_encoders[col] = le
    # Save label_encoders to disk
    with open(save_path, 'wb') as f:
        pickle.dump(label_encoders, f)

    return df,label_encoders

# Load your trained LabelEncoders
def load_label_encoders(path: str) -> dict:
    with open(path, 'rb') as f:
        return pickle.load(f)

# Transform data using fitted LabelEncoders
def transform_label_data(df: pd.DataFrame, label_encoders: dict) -> pd.DataFrame:
    for col in df.columns:
        if col in label_encoders.keys():
            # Transform the training dataset
            df[col] = label_encoders[col].transform(df[col].astype(str))

    return df, label_encoders

# Example usage (commented out)
# train_transformed = transform_label_data(train_df, load_label_encoders('label_encoder.pkl'))

"""

"import pandas as pd\nfrom sklearn.preprocessing import LabelEncoder\nimport pickle\n\ndef fit_label_encoders(df: pd.DataFrame, save_path: str) -> dict:\n    label_encoders = {}\n    for col in df.columns:\n        if df[col].dtype == 'object' and col != 'RainTomorrow':\n            le = LabelEncoder()\n            le.fit(df[col].astype(str))\n            label_encoders[col] = le\n    # Save label_encoders to disk\n    with open(save_path, 'wb') as f:\n        pickle.dump(label_encoders, f)\n    \n    return df,label_encoders\n\n# Load your trained LabelEncoders\ndef load_label_encoders(path: str) -> dict:\n    with open(path, 'rb') as f:\n        return pickle.load(f)\n\n# Transform data using fitted LabelEncoders\ndef transform_label_data(df: pd.DataFrame, label_encoders: dict) -> pd.DataFrame:\n    for col in df.columns:\n        if col in label_encoders.keys():\n            # Transform the training dataset\n            df[col] = label_encoders[col].transform(df[col].astype(str))\n 

In [ ]:
"""import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

def fit_label_encoders(df: pd.DataFrame, save_path: str) -> (pd.DataFrame, dict):
    label_encoders = {}
    for col in df.columns:
        if df[col].dtype == 'object' and col != 'RainTomorrow':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le
    # Save label_encoders to disk
    with open(save_path, 'wb') as f:
        pickle.dump(label_encoders, f)

    return df, label_encoders

# Example usage
# df_encoded, LE = fit_label_encoders(df2, 'label_encoder.pkl')
"""

"import pandas as pd\nfrom sklearn.preprocessing import LabelEncoder\nimport pickle\n\ndef fit_label_encoders(df: pd.DataFrame, save_path: str) -> (pd.DataFrame, dict):\n    label_encoders = {}\n    for col in df.columns:\n        if df[col].dtype == 'object' and col != 'RainTomorrow':\n            le = LabelEncoder()\n            df[col] = le.fit_transform(df[col].astype(str))\n            label_encoders[col] = le\n    # Save label_encoders to disk\n    with open(save_path, 'wb') as f:\n        pickle.dump(label_encoders, f)\n    \n    return df, label_encoders\n\n# Example usage\n# df_encoded, LE = fit_label_encoders(df2, 'label_encoder.pkl')\n"

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle

def fit_label_encoders(df: pd.DataFrame, save_path: str) -> (pd.DataFrame, dict):
    label_encoders = {}
    for col in df.columns:
        if df[col].dtype == 'object' and col != 'RainTomorrow':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col].astype(str))
            label_encoders[col] = le
    # Save label_encoders to disk
    with open(save_path, 'wb') as f:
        pickle.dump(label_encoders, f)

    return df, label_encoders

def load_label_encoders(path: str) -> dict:
    with open(path, 'rb') as f:
        return pickle.load(f)

def transform_label_data(df: pd.DataFrame, label_encoders: dict) -> pd.DataFrame:
    for col, le in label_encoders.items():
        df[col] = le.transform(df[col].astype(str))
    return df

def load_and_transform_new_data(df: pd.DataFrame, encoder_path: str) -> pd.DataFrame:
    label_encoders = load_label_encoders(encoder_path)
    df_transformed = transform_label_data(df, label_encoders)
    return df_transformed

# Example usage
df_encoded, LE = fit_label_encoders(df2, 'label_encoder.pkl')
df_inference_encoded = load_and_transform_new_data(df_inference, 'label_encoder.pkl')


In [ ]:
# df_encoded, LE = fit_label_encoders(df2, 'label_encoder.pkl')


In [ ]:
# df_inference_encoded= load_and_transform_new_data(df_inference,'label_encoder.pkl')

In [ ]:
LE

{'Location': LabelEncoder(),
 'WindGustDir': LabelEncoder(),
 'WindDir9am': LabelEncoder(),
 'WindDir3pm': LabelEncoder(),
 'RainToday': LabelEncoder()}

In [ ]:
type(LE)

dict

In [ ]:
df_inference_encoded.head(1)

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,year,month,day
14,2,8.4,24.6,0.0,5.370076,6.796154,3,43.012753,8,14,...,1008.7,5.32529,5.083749,15.9,23.5,0,NaN,2008,12,15


In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142193 entries, 0 to 145458
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  int64  
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   Evaporation    142193 non-null  float64
 5   Sunshine       142193 non-null  float64
 6   WindGustDir    142193 non-null  int64  
 7   WindGustSpeed  142193 non-null  float64
 8   WindDir9am     142193 non-null  int64  
 9   WindDir3pm     142193 non-null  int64  
 10  WindSpeed9am   142193 non-null  float64
 11  WindSpeed3pm   142193 non-null  float64
 12  Humidity9am    142193 non-null  float64
 13  Humidity3pm    142193 non-null  float64
 14  Pressure9am    142193 non-null  float64
 15  Pressure3pm    142193 non-null  float64
 16  Cloud9am       142193 non-null  float64
 17  Cloud3pm       142193 non-null  fl

In [ ]:
def convert_to_float(df: pd.DataFrame) -> pd.DataFrame:
    for col in df.columns:
        if col != 'RainTomorrow':
            df[col] = df[col].astype(float)
    return df

In [ ]:
df_float=convert_to_float(df_encoded)

In [ ]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 142193 entries, 0 to 145458
Data columns (total 25 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Location       142193 non-null  float64
 1   MinTemp        142193 non-null  float64
 2   MaxTemp        142193 non-null  float64
 3   Rainfall       142193 non-null  float64
 4   Evaporation    142193 non-null  float64
 5   Sunshine       142193 non-null  float64
 6   WindGustDir    142193 non-null  float64
 7   WindGustSpeed  142193 non-null  float64
 8   WindDir9am     142193 non-null  float64
 9   WindDir3pm     142193 non-null  float64
 10  WindSpeed9am   142193 non-null  float64
 11  WindSpeed3pm   142193 non-null  float64
 12  Humidity9am    142193 non-null  float64
 13  Humidity3pm    142193 non-null  float64
 14  Pressure9am    142193 non-null  float64
 15  Pressure3pm    142193 non-null  float64
 16  Cloud9am       142193 non-null  float64
 17  Cloud3pm       142193 non-null  fl

In [ ]:
df_infer_float=convert_to_float(df_inference_encoded)

In [ ]:
import pandas as pd

# Filter extreme outliers
def drop_extreme_outliers(df: pd.DataFrame) -> pd.DataFrame:
    numeric_df = df.select_dtypes(include=['number'])

    # Calculate 0.5th and 99.5th percentiles for each numeric column
    Q01 = numeric_df.quantile(0.005)
    Q99 = numeric_df.quantile(0.995)

    # Define lower and upper bounds for numeric columns
    lower_bound = Q01 - 1.5 * (Q99 - Q01)
    upper_bound = Q99 + 1.5 * (Q99 - Q01)

    # Identify outliers for each numeric column
    outliers = pd.DataFrame(False, index=df.index, columns=numeric_df.columns)
    for col in numeric_df.columns:
        outliers[col] = (numeric_df[col] < lower_bound[col]) | (numeric_df[col] > upper_bound[col])

    # Remove rows where any numeric column has outliers
    filtered_df = df[~outliers.any(axis=1)]
    return filtered_df

# Example usage
# filtered_df = drop_extreme_outliers(df)


In [ ]:
df_float_filtered=drop_extreme_outliers(df_float)

In [ ]:
print(df_float.shape)
print(df_float_filtered.shape)

(142193, 25)
(142071, 25)


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def min_max_scale_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    scaler = MinMaxScaler()

    # separate the feature from the target column
    features = df.drop('RainTomorrow', axis=1)
    target = df['RainTomorrow']

    # scale the feature
    scaled_features = scaler.fit_transform(features)

    # reassemble the dataframe
    scaled_df = pd.DataFrame(scaled_features, columns=features.columns, index=df.index)
    scaled_df['RainTomorrow'] = target

    return scaled_df

    df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
    return df_scaled

In [ ]:
df_scaled=min_max_scale_dataframe(df_float_filtered)
df_infer_scaled=min_max_scale_dataframe(df_inference_encoded)

In [ ]:
df_scaled.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,year,month,day,RainTomorrow
0,0.041667,0.516509,0.523629,0.004566,0.115886,0.525852,0.866667,0.294574,0.866667,0.933333,...,0.4800,0.888889,0.500352,0.508439,0.522073,0.0,0.1,1.0,0.000000,No
1,0.041667,0.375000,0.565217,0.000000,0.115886,0.525852,0.933333,0.294574,0.400000,1.000000,...,0.4912,0.493021,0.500352,0.514768,0.570058,0.0,0.1,1.0,0.033333,No
2,0.041667,0.504717,0.576560,0.000000,0.115886,0.525852,1.000000,0.310078,0.866667,1.000000,...,0.5056,0.493021,0.222222,0.594937,0.548944,0.0,0.1,1.0,0.066667,No
3,0.041667,0.417453,0.620038,0.000000,0.115886,0.525852,0.266667,0.139535,0.600000,0.000000,...,0.5712,0.493021,0.500352,0.533755,0.612284,0.0,0.1,1.0,0.100000,No
4,0.041667,0.613208,0.701323,0.007610,0.115886,0.525852,0.866667,0.271318,0.066667,0.466667,...,0.4624,0.777778,0.888889,0.527426,0.673704,0.0,0.1,1.0,0.133333,No


In [ ]:
df_scaled.shape

(142071, 25)

In [ ]:
def split_feature_target(df,target_column):
    X = df.drop(target_column, axis=1)
    y = df[target_column]
    return X, y

In [ ]:
X, y = split_feature_target(df_scaled, 'RainTomorrow')

In [ ]:
X.shape

(142071, 24)

In [ ]:
y.shape

(142071,)

In [ ]:
X_infer, y_infer = split_feature_target(df_inference_encoded, 'RainTomorrow')

In [ ]:
def perform_train_test_split(X, y, test_size=0.2, random_state=42):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test=perform_train_test_split(X,y)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(113656, 24)
(28415, 24)
(113656,)
(28415,)


In [ ]:
def train_logistic_regression_model(X_train, X_test, y_train, y_test):
    logreg = LogisticRegression(solver='liblinear',random_state=0)
    logreg.fit(X_train, y_train)

    y_pred_train = logreg.predict(X_train)
    # print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))
    y_pred_test = logreg.predict(X_test)


    return y_pred_train,y_pred_test, logreg

In [ ]:
y_pred_train,y_pred_test,clf=train_logistic_regression_model(X_train, X_test, y_train, y_test)

In [ ]:
def print_accuracy_score(y_pred_train,y_pred_test, y_train, y_test):
    print('Accuracy of logistic regression classifier on train set: {:.4f}'.format(clf.score(X_train, y_train)))
    print('Accuracy of logistic regression classifier on test set: {:.4f}'.format(clf.score(X_test, y_test)))

In [ ]:
print_accuracy_score(y_pred_train,y_pred_test, y_train, y_test)

Accuracy of logistic regression classifier on train set: 0.8427
Accuracy of logistic regression classifier on test set: 0.8422


In [ ]:
def print_classification_matrics(y_true,y_pred):
    print(classification_report(y_true, y_pred))

    cm=confusion_matrix(y_true, y_pred)
    print(cm)

    TP=cm[0][0]
    TN=cm[1][1]
    FP=cm[0][1]
    FN=cm[1][0]

    # true positive, true negative, false positive, false negative
    tp,tn, fp, fn,  = confusion_matrix(y_true, y_pred).ravel()
    print('True Positive(TP)  = ', tp)
    print('False Positive(FP) = ', fp)
    print('True Negative(TN)  = ', tn)
    print('False Negative(FN) = ', fn)

    # classification accuracy
    classification_accuracy = (tp + tn) / (tp + fp + tn + fn)
    print('Classification accuracy = ', classification_accuracy)
    # classification error
    classification_error = (fp + fn) / (tp + fp + tn + fn)
    print('Classification error = ', classification_error)
    # precision
    precision = tp / (tp + fp)
    print('Precision = ', precision)
    # recall/sensitivity
    recall = tp / (tp + fn)
    print('Recall = ', recall)
    True_Positive_Rate = tp / (tp + fn)
    print('True Positive Rate = ', True_Positive_Rate)
    # specificity
    specificity = tn / (tn + fp)
    print('Specificity = ', specificity)

In [ ]:
print_classification_matrics(y_test,y_pred_test)

              precision    recall  f1-score   support

          No       0.87      0.94      0.90     22034
         Yes       0.72      0.49      0.58      6381

    accuracy                           0.84     28415
   macro avg       0.79      0.72      0.74     28415
weighted avg       0.83      0.84      0.83     28415

[[20786  1248]
 [ 3235  3146]]
True Positive(TP)  =  20786
False Positive(FP) =  3235
True Negative(TN)  =  1248
False Negative(FN) =  3146
Classification accuracy =  0.7754355094140419
Classification error =  0.22456449058595812
Precision =  0.8653261729320179
Recall =  0.8685442085910079
True Positive Rate =  0.8685442085910079
Specificity =  0.27838501003792104


In [ ]:
import pickle

with open('logistic_regression_model.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)


In [ ]:
y_pred_infer=clf.predict(X_infer)

In [ ]:
y_pred_infer

array(['No', 'No', 'No', ..., 'No', 'No', 'No'], dtype=object)

In [ ]:
# concatenate y_pred_infer to df_inference
df_inference['RainTomorrow_predicted'] = y_pred_infer

# drop the original raintomorrow col
df_inference.drop('RainTomorrow', axis=1, inplace=True)


In [ ]:
df_inference.head(4)

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,year,month,day,RainTomorrow_predicted
14,2.0,8.4,24.600000,0.0,5.370076,6.796154,3.0,43.012753,8.0,14.0,...,1008.7,5.32529,5.083749,15.9,23.5,0.0,2008.0,12.0,15.0,No
283,2.0,2.6,22.890077,0.0,5.370076,6.796154,3.0,43.012753,0.0,14.0,...,1017.6,5.32529,6.000000,9.7,14.3,0.0,2009.0,9.0,10.0,No
435,2.0,22.1,35.100000,0.0,5.370076,6.796154,3.0,43.012753,1.0,12.0,...,1017.8,5.32529,1.000000,26.9,33.5,0.0,2010.0,2.0,9.0,No
437,2.0,21.5,35.000000,0.0,5.370076,6.796154,3.0,43.012753,10.0,0.0,...,1010.3,5.32529,8.000000,24.2,33.1,0.0,2010.0,2.0,11.0,No
